In [1]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
from misc_functions import load_sig_chans, channel_names_to_indices


['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize the outer dictionary.
subjects_electrodestoROIs_dict = {}

### make subjects rois to electrodes dict. Don't need to run this more than once.

In [49]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094']

# subjects = ['D0090'] #testing cuz d0065 being weird

for sub in subjects:
    # sub = 'D0059'
    task = 'GlobalLocal'
    output_name = "Response_fixationCrossBase_1sec_mirror"
    events = ["Response"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False


    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    default_dict = gen_labels(good.info)
    
    # Create rawROI_dict for the subject
    rawROI_dict = defaultdict(list)
    for key, value in default_dict.items():
        rawROI_dict[value].append(key)
    rawROI_dict = dict(rawROI_dict)

    # Filter out keys containing "White-Matter"
    filtROI_dict = {key: value for key, value in rawROI_dict.items() if "White-Matter" not in key}

    # Store the dictionaries in the subjects dictionary
    subjects_electrodestoROIs_dict[sub] = {
        'default_dict': dict(default_dict),
        'rawROI_dict': dict(rawROI_dict),
        'filtROI_dict': dict(filtROI_dict)
    }


# # Save to a JSON file. Uncomment when actually running.
filename = 'subjects_electrodestoROIs_dict.json'
with open(filename, 'w') as file:
    json.dump(subjects_electrodestoROIs_dict, file, indent=4)

print(f"Saved subjects_dict to {filename}")

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_d

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


good channels before dropping bads: 178
filt channels before dropping bads: 178


### load subjects electrodes to rois dict

In [ ]:
# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

Loaded data from subjects_electrodestoROIs_dict.json


### load high gamma data so we can do roi analysis on it
once we have more subjects, turn this into a function and loop over all subjects.  
this code is a crime against humanity

In [ ]:
import os
import mne
import numpy as np


def load_mne_objects(sub, output_name, task, LAB_root=None):
    """
    Load MNE objects for a given subject and output name.

    Parameters:
    - sub (str): Subject identifier.
    - output_name (str): Output name used in the file naming.
    - task (str): Task identifier.
    - LAB_root (str, optional): Root directory for the lab. If None, it will be determined based on the OS.

    Returns:
    A dictionary containing loaded MNE objects.
    """

    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Define file paths
    HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
    HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
    HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

    # Load the objects
    HG_ev1 = mne.read_epochs(HG_ev1_file)
    HG_base = mne.read_epochs(HG_base_file)
    HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    return {
        'HG_ev1': HG_ev1,
        'HG_base': HG_base,
        'HG_ev1_rescaled': HG_ev1_rescaled,
        'HG_ev1_evoke': HG_ev1_evoke,
        'HG_ev1_evoke_rescaled': HG_ev1_evoke_rescaled
    }

# Example usage
# sub = 'D0057'
# output_name = "Stimulus_i25and75_fixationCrossBase_1sec_mirror"
# task = 'GlobalLocal'
loaded_objects_D0057_i = load_mne_objects('D0057', "Stimulus_c25_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0057_c = load_mne_objects('D0057', "Stimulus_c75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

# loaded_objects_D0059_i = load_mne_objects('D0059', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
# loaded_objects_D0059_c = load_mne_objects('D0059', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

# loaded_objects_D0063_i = load_mne_objects('D0063', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
# loaded_objects_D0063_c = load_mne_objects('D0063', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

# loaded_objects_D0065_i = load_mne_objects('D0065', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
# loaded_objects_D0065_c = load_mne_objects('D0065', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

# loaded_objects_D0069_i = load_mne_objects('D0069', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
# loaded_objects_D0069_c = load_mne_objects('D0069', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

# loaded_objects_D0071_i = load_mne_objects('D0071', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
# loaded_objects_D0071_c = load_mne_objects('D0071', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')


# Access the objects
HG_ev1_D0057_i = loaded_objects_D0057_i['HG_ev1']
HG_base_D0057_i = loaded_objects_D0057_i['HG_base']
HG_ev1_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_rescaled']
HG_ev1_evoke_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke_rescaled']

HG_ev1_D0057_c = loaded_objects_D0057_c['HG_ev1']
HG_base_D0057_c = loaded_objects_D0057_c['HG_base']
HG_ev1_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_rescaled']
HG_ev1_evoke_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0059_i = loaded_objects_D0059_i['HG_ev1']
# HG_base_D0059_i = loaded_objects_D0059_i['HG_base']
# HG_ev1_rescaled_D0059_i = loaded_objects_D0059_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0059_i = loaded_objects_D0059_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0059_i = loaded_objects_D0059_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0059_c = loaded_objects_D0059_c['HG_ev1']
# HG_base_D0059_c = loaded_objects_D0059_c['HG_base']
# HG_ev1_rescaled_D0059_c = loaded_objects_D0059_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0059_c = loaded_objects_D0059_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0059_c = loaded_objects_D0059_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0063_i = loaded_objects_D0063_i['HG_ev1']
# HG_base_D0063_i = loaded_objects_D0063_i['HG_base']
# HG_ev1_rescaled_D0063_i = loaded_objects_D0063_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0063_i = loaded_objects_D0063_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0063_i = loaded_objects_D0063_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0063_c = loaded_objects_D0063_c['HG_ev1']
# HG_base_D0063_c = loaded_objects_D0063_c['HG_base']
# HG_ev1_rescaled_D0063_c = loaded_objects_D0063_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0063_c = loaded_objects_D0063_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0063_c = loaded_objects_D0063_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0065_i = loaded_objects_D0065_i['HG_ev1']
# HG_base_D0065_i = loaded_objects_D0065_i['HG_base']
# HG_ev1_rescaled_D0065_i = loaded_objects_D0065_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0065_i = loaded_objects_D0065_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0065_i = loaded_objects_D0065_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0065_c = loaded_objects_D0065_c['HG_ev1']
# HG_base_D0065_c = loaded_objects_D0065_c['HG_base']
# HG_ev1_rescaled_D0065_c = loaded_objects_D0065_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0065_c = loaded_objects_D0065_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0065_c = loaded_objects_D0065_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0069_i = loaded_objects_D0069_i['HG_ev1']
# HG_base_D0069_i = loaded_objects_D0069_i['HG_base']
# HG_ev1_rescaled_D0069_i = loaded_objects_D0069_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0069_i = loaded_objects_D0069_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0069_i = loaded_objects_D0069_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0069_c = loaded_objects_D0069_c['HG_ev1']
# HG_base_D0069_c = loaded_objects_D0069_c['HG_base']
# HG_ev1_rescaled_D0069_c = loaded_objects_D0069_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0069_c = loaded_objects_D0069_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0069_c = loaded_objects_D0069_c['HG_ev1_evoke_rescaled']

# HG_ev1_D0071_i = loaded_objects_D0071_i['HG_ev1']
# HG_base_D0071_i = loaded_objects_D0071_i['HG_base']
# HG_ev1_rescaled_D0071_i = loaded_objects_D0071_i['HG_ev1_rescaled']
# HG_ev1_evoke_D0071_i = loaded_objects_D0071_i['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0071_i = loaded_objects_D0071_i['HG_ev1_evoke_rescaled']

# HG_ev1_D0071_c = loaded_objects_D0071_c['HG_ev1']
# HG_base_D0071_c = loaded_objects_D0071_c['HG_base']
# HG_ev1_rescaled_D0071_c = loaded_objects_D0071_c['HG_ev1_rescaled']
# HG_ev1_evoke_D0071_c = loaded_objects_D0071_c['HG_ev1_evoke']
# HG_ev1_evoke_rescaled_D0071_c = loaded_objects_D0071_c['HG_ev1_evoke_rescaled']

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
168 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.

### load evoked and stuff for all subjects in a dictionary

In [ ]:
import os
import mne
import numpy as np

def create_subjects_mne_objects_dict(subjects, output_names, task, LAB_root=None):
    subjects_mne_objects = {}

    for sub in subjects:
        print(f"Loading data for subject: {sub}")  # Debugging print
        sub_mne_objects = {}
        for output_name in output_names:
            print(f"  Loading output: {output_name}")  # Debugging print
            mne_objects = load_mne_objects(sub, output_name, task, LAB_root)

            # Debugging prints for data shapes
            print(f"    HG_ev1 shape: {mne_objects['HG_ev1'].get_data().shape}")
            print(f"    HG_base shape: {mne_objects['HG_base'].get_data().shape}")
            print(f"    HG_ev1_rescaled shape: {mne_objects['HG_ev1_rescaled'].get_data().shape}")
            print(f"    HG_ev1_evoke shape: {mne_objects['HG_ev1_evoke'].data.shape}")
            print(f"    HG_ev1_evoke_rescaled shape: {mne_objects['HG_ev1_evoke_rescaled'].data.shape}")

            sub_mne_objects[output_name] = mne_objects
        subjects_mne_objects[sub] = sub_mne_objects

    return subjects_mne_objects

# Example usage
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094']
output_names = ["Stimulus_c25and75_fixationCrossBase_1sec_mirror", "Stimulus_i25and75_fixationCrossBase_1sec_mirror"]
task = 'GlobalLocal'

subjects_mne_objects = create_subjects_mne_objects_dict(subjects, output_names, task)

Loading data for subject: D0057
  Loading output: Stimulus_c25and75_fixationCrossBase_1sec_mirror
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25and75_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
224 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25and75_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25and

C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1 shape: {mne_objects['HG_ev1'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_base shape: {mne_objects['HG_base'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1_rescaled shape: {mne_objects['HG_ev1_rescaled'].get_data().shape}")


    HG_ev1 shape: (224, 175, 2561)
    HG_base shape: (448, 175, 1025)
    HG_ev1_rescaled shape: (224, 175, 5121)
    HG_ev1_evoke shape: (175, 2561)
    HG_ev1_evoke_rescaled shape: (175, 5121)
  Loading output: Stimulus_i25and75_fixationCrossBase_1sec_mirror
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
224 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline

C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1 shape: {mne_objects['HG_ev1'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_base shape: {mne_objects['HG_base'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1_rescaled shape: {mne_objects['HG_ev1_rescaled'].get_data().shape}")


    HG_ev1 shape: (224, 175, 2561)
    HG_base shape: (448, 175, 1025)
    HG_ev1_rescaled shape: (224, 175, 5121)
    HG_ev1_evoke shape: (175, 2561)
    HG_ev1_evoke_rescaled shape: (175, 5121)
Loading data for subject: D0059
  Loading output: Stimulus_c25and75_fixationCrossBase_1sec_mirror
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0059\D0059_Stimulus_c25and75_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
224 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0059\D0059_Stimulus_c25and75_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 m

...

C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1 shape: {mne_objects['HG_ev1'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_base shape: {mne_objects['HG_base'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1_rescaled shape: {mne_objects['HG_ev1_rescaled'].get_data().shape}")


    HG_ev1 shape: (224, 157, 2561)
    HG_base shape: (448, 157, 1025)
    HG_ev1_rescaled shape: (224, 157, 5121)
    HG_ev1_evoke shape: (157, 2561)
    HG_ev1_evoke_rescaled shape: (157, 5121)
Loading data for subject: D0094
  Loading output: Stimulus_c25and75_fixationCrossBase_1sec_mirror
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0094\D0094_Stimulus_c25and75_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
224 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0094\D0094_Stimulus_c25and75_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 m

C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1 shape: {mne_objects['HG_ev1'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_base shape: {mne_objects['HG_base'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1_rescaled shape: {mne_objects['HG_ev1_rescaled'].get_data().shape}")


    HG_ev1 shape: (224, 230, 2561)
    HG_base shape: (448, 230, 1025)
    HG_ev1_rescaled shape: (224, 230, 5121)
    HG_ev1_evoke shape: (230, 2561)
    HG_ev1_evoke_rescaled shape: (230, 5121)
  Loading output: Stimulus_i25and75_fixationCrossBase_1sec_mirror
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0094\D0094_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
224 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0094\D0094_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline

C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1 shape: {mne_objects['HG_ev1'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_base shape: {mne_objects['HG_base'].get_data().shape}")
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1585441012.py:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"    HG_ev1_rescaled shape: {mne_objects['HG_ev1_rescaled'].get_data().shape}")


### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [42]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# List of subjects
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094']

# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094


### get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [43]:
def filter_electrodes_by_roi(subjects_electrodes_dict, sig_chans_per_subject, roi_list):
    """
    Filters electrodes based on specified ROIs and returns significant electrodes for each subject.

    Args:
    subjects_electrodes_dict (dict): A dictionary with subjects as keys and electrode-to-ROI mappings as values.
    sig_chans_per_subject (dict): A dictionary with subjects as keys and lists of significant channels as values.
    roi_list (list): A list of ROIs to filter electrodes.

    Returns:
    dict: A dictionary with subjects as keys and lists of significant electrodes in specified ROIs as values.
    """
    filtered_electrodes_per_subject = {}

    for sub, electrodes_dict in subjects_electrodes_dict.items():
        filtered = {key: value for key, value in electrodes_dict['filtROI_dict'].items() 
                    if any(roi in key for roi in roi_list)}

        # Aggregate electrodes into a list for each subject
        filtered_electrodes = []
        for electrodes in filtered.values():
            filtered_electrodes.extend(electrodes)

        filtered_electrodes_per_subject[sub] = filtered_electrodes
        print(f'For subject {sub}, {", ".join(roi_list)} electrodes are: {filtered_electrodes}')

    # Now filter for significant electrodes
    sig_filtered_electrodes_per_subject = {}

    for sub, filtered_electrodes in filtered_electrodes_per_subject.items():
        # Retrieve the list of significant channels for the subject
        sig_chans = sig_chans_per_subject.get(sub, [])

        # Find the intersection of filtered electrodes and significant channels for the subject
        sig_filtered_electrodes = [elec for elec in filtered_electrodes if elec in sig_chans]

        # Store the significant filtered electrodes for the subject
        sig_filtered_electrodes_per_subject[sub] = sig_filtered_electrodes
        print(f"Subject {sub} significant {', '.join(roi_list)} electrodes: {sig_filtered_electrodes}")

    return filtered_electrodes_per_subject, sig_filtered_electrodes_per_subject

# Example usage:
dlpfc_rois = ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"] #dorsolateral prefrontal cortex
acc_rois = ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"] #anterior cingulate cortex
parietal_rois = ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"] #superior parietal lobule, intraparietal sulcus, and inferior parietal lobule (split into angular gyrus and supramarginal gyrus)
dlpfc_electrodes_per_subject, sig_dlpfc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, dlpfc_rois)
acc_electrodes_per_subject, sig_acc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, acc_rois)
parietal_electrodes_per_subject, sig_parietal_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, parietal_rois)

For subject D0057, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12']
For subject D0059, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LASF10', 'LASF14', 'LASF15', 'LASF16', 'LMSF5', 'LMSF6', 'LMSF12', 'LPSF10', 'LPSF12', 'RAI10', 'RAI11', 'RAI16', 'RAMF11', 'RAMF12', 'RAMF13', 'RMMF13', 'RMMF14', 'RASF15', 'RMSF8', 'RMSF9', 'RMSF10', 'RMSF7', 'RAMF8', 'RAMF9', 'RAMF10', 'RMMF9', 'RMMF10']
For subject D0065, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RASF13', 'RASF14', 'RASF15', 'RMSF11', 'RMSF12', 'RMSF13', 'RMSF14']
For subject D0069, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: []
For subject D0071, G_fron

get total number of electrodes (make this modular with roi later once everything works)

In [44]:
total_entries = 0
for sub in sig_dlpfc_electrodes_per_subject:
    # Since each subject's entry is a list, directly add its length
    total_entries += len(sig_dlpfc_electrodes_per_subject[sub])

print("Total number of sig dlpfc electrodes across all subjects:", total_entries)

total_entries = 0
for sub in sig_acc_electrodes_per_subject:
    # Since each subject's entry is a list, directly add its length
    total_entries += len(sig_acc_electrodes_per_subject[sub])

print("Total number of sig acc electrodes across all subjects:", total_entries)

total_entries = 0
for sub in sig_parietal_electrodes_per_subject:
    # Since each subject's entry is a list, directly add its length
    total_entries += len(sig_parietal_electrodes_per_subject[sub])

print("Total number of sig parietal electrodes across all subjects:", total_entries)

Total number of sig dlpfc electrodes across all subjects: 17
Total number of sig acc electrodes across all subjects: 4
Total number of sig parietal electrodes across all subjects: 13


### do stats

current approach is to run time_perm_cluster on significant dlpfc electrodes for each subject, comparing congruent and incongruent conditions. Then, average p-values across all subjects. Discuss this with Greg, probably wrong approach.

**1/23 new approach is to average across all trials for sig dlpfc electrodes, comparing incongruent and congruent conditions. Then, run stats on this new avg electrode value x time array.

Also, I'm using HG_ev1_rescaled instead of HG_ev1 to compare congruent and incongruent, so that they're normalized with a common baseline. I think this is better than comparing the raw HG traces directly.

### this is 1/23 new approach of avg across trials first

do stats and plotting together. Stats needs trial avg data, plotting just needs congruent_data without trial averaging (initially at least)  
this code is so bad right now, turn into a function later  

trialAvg is for the time perm cluster stats  
timeAvg is for the window stats (not sure if this is even right)



In [45]:
import numpy as np

# Initialize lists to store data
output_0_data_trialAvg_list_dlpfc = []
output_1_data_trialAvg_list_dlpfc = []
output_0_data_trialAvg_list_acc = []
output_1_data_trialAvg_list_acc = []
output_0_data_trialAvg_list_parietal = []
output_1_data_trialAvg_list_parietal = []

# Initialize lists to store time-averaged data
output_0_data_timeAvg_list_dlpfc = []
output_1_data_timeAvg_list_dlpfc = []
output_0_data_timeAvg_list_acc = []
output_1_data_timeAvg_list_acc = []
output_0_data_timeAvg_list_parietal = []
output_1_data_timeAvg_list_parietal = []

start_idx, end_idx = 2048, 3072  # Time window from 0 to 0.5 seconds

subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094']

for sub in subjects:
    # Skip this subject if no dlpfc electrodes
    if not sig_dlpfc_electrodes_per_subject[sub]:
        continue

    # Load trial-level data for each condition and pick significant DLPFC electrodes
    output_0_epochs = subjects_mne_objects[sub][output_names[0]]['HG_ev1_rescaled'].copy().pick_channels(sig_dlpfc_electrodes_per_subject[sub])
    output_1_epochs = subjects_mne_objects[sub][output_names[1]]['HG_ev1_rescaled'].copy().pick_channels(sig_dlpfc_electrodes_per_subject[sub])

    # Average across the trials dimension, ignoring NaNs
    output_0_data_trialAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data(), axis=0))
    output_1_data_trialAvg_list_dlpfc.append(np.nanmean(output_1_epochs.get_data(), axis=0))

    # average from t=0 to t=0.5, assuming sampling freq of 2048 hz
    output_0_data_timeAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data()[:, :, start_idx:end_idx], axis=2))
    output_1_data_timeAvg_list_dlpfc.append(np.nanmean(output_1_epochs.get_data()[:, :, start_idx:end_idx], axis=2))

# do for acc
for sub in subjects:
    # Skip this subject if no dlpfc electrodes
    if not sig_acc_electrodes_per_subject[sub]:
        continue

    # Load trial-level data for each condition and pick significant DLPFC electrodes
    output_0_epochs = subjects_mne_objects[sub][output_names[0]]['HG_ev1_rescaled'].copy().pick_channels(sig_acc_electrodes_per_subject[sub])
    output_1_epochs = subjects_mne_objects[sub][output_names[1]]['HG_ev1_rescaled'].copy().pick_channels(sig_acc_electrodes_per_subject[sub])

    # Average across the trials dimension, ignoring NaNs
    output_0_data_trialAvg_list_acc.append(np.nanmean(output_0_epochs.get_data(), axis=0))
    output_1_data_trialAvg_list_acc.append(np.nanmean(output_1_epochs.get_data(), axis=0))
    output_0_data_timeAvg_list_acc.append(np.nanmean(output_0_epochs.get_data()[:, :, start_idx:end_idx], axis=2))
    output_1_data_timeAvg_list_acc.append(np.nanmean(output_1_epochs.get_data()[:, :, start_idx:end_idx], axis=2))

#do for parietal
for sub in subjects:
    # Skip this subject if no dlpfc electrodes
    if not sig_parietal_electrodes_per_subject[sub]:
        continue

    # Load trial-level data for each condition and pick significant DLPFC electrodes
    output_0_epochs = subjects_mne_objects[sub][output_names[0]]['HG_ev1_rescaled'].copy().pick_channels(sig_parietal_electrodes_per_subject[sub])
    output_1_epochs = subjects_mne_objects[sub][output_names[1]]['HG_ev1_rescaled'].copy().pick_channels(sig_parietal_electrodes_per_subject[sub])

    # Average across the trials dimension, ignoring NaNs
    output_0_data_trialAvg_list_parietal.append(np.nanmean(output_0_epochs.get_data(), axis=0))
    output_1_data_trialAvg_list_parietal.append(np.nanmean(output_1_epochs.get_data(), axis=0))

    output_0_data_timeAvg_list_parietal.append(np.nanmean(output_0_epochs.get_data()[:, :, start_idx:end_idx], axis=2))
    output_1_data_timeAvg_list_parietal.append(np.nanmean(output_1_epochs.get_data()[:, :, start_idx:end_idx], axis=2))

# below functions only go this far, keep this code
# Concatenate data across all electrodes
output_0_data_trialAvg_dlpfc = np.concatenate(output_0_data_trialAvg_list_dlpfc, axis=0)
output_1_data_trialAvg_dlpfc = np.concatenate(output_1_data_trialAvg_list_dlpfc, axis=0)

output_0_data_trialAvg_acc = np.concatenate(output_0_data_trialAvg_list_acc, axis=0)
output_1_data_trialAvg_acc = np.concatenate(output_1_data_trialAvg_list_acc, axis=0)

output_0_data_trialAvg_parietal = np.concatenate(output_0_data_trialAvg_list_parietal, axis=0)
output_1_data_trialAvg_parietal = np.concatenate(output_1_data_trialAvg_list_parietal, axis=0)


# Concatenate time-averaged data across all subjects for each brain region and condition
output_0_data_timeAvg_dlpfc = np.concatenate(output_0_data_timeAvg_list_dlpfc, axis=1)
output_1_data_timeAvg_dlpfc = np.concatenate(output_1_data_timeAvg_list_dlpfc, axis=1)

output_0_data_timeAvg_acc = np.concatenate(output_0_data_timeAvg_list_acc, axis=1)
output_1_data_timeAvg_acc = np.concatenate(output_1_data_timeAvg_list_acc, axis=1)

output_0_data_timeAvg_parietal = np.concatenate(output_0_data_timeAvg_list_parietal, axis=1)
output_1_data_timeAvg_parietal = np.concatenate(output_1_data_timeAvg_list_parietal, axis=1)


# Calculate mean and SEM across electrodes
overall_average_dlpfc_output_0 = np.nanmean(output_0_data_trialAvg_dlpfc, axis=0)
overall_sem_dlpfc_output_0 = np.std(output_0_data_trialAvg_dlpfc, axis=0, ddof=1) / np.sqrt(output_0_data_trialAvg_dlpfc.shape[0])
overall_average_dlpfc_output_1 = np.nanmean(output_1_data_trialAvg_dlpfc, axis=0)
overall_sem_dlpfc_output_1 = np.std(output_1_data_trialAvg_dlpfc, axis=0, ddof=1) / np.sqrt(output_1_data_trialAvg_dlpfc.shape[0])

overall_average_acc_output_0 = np.nanmean(output_0_data_trialAvg_acc, axis=0)
overall_sem_acc_output_0 = np.std(output_0_data_trialAvg_acc, axis=0, ddof=1) / np.sqrt(output_0_data_trialAvg_acc.shape[0])
overall_average_acc_output_1 = np.nanmean(output_1_data_trialAvg_acc, axis=0)
overall_sem_acc_output_1 = np.std(output_1_data_trialAvg_acc, axis=0, ddof=1) / np.sqrt(output_1_data_trialAvg_acc.shape[0])

overall_average_parietal_output_0 = np.nanmean(output_0_data_trialAvg_parietal, axis=0)
overall_sem_parietal_output_0 = np.std(output_0_data_trialAvg_parietal, axis=0, ddof=1) / np.sqrt(output_0_data_trialAvg_parietal.shape[0])
overall_average_parietal_output_1 = np.nanmean(output_1_data_trialAvg_parietal, axis=0)
overall_sem_parietal_output_1 = np.std(output_1_data_trialAvg_parietal, axis=0, ddof=1) / np.sqrt(output_1_data_trialAvg_parietal.shape[0])


# Run the permutation test
mat_dlpfc = time_perm_cluster(output_0_data_trialAvg_dlpfc, output_1_data_trialAvg_dlpfc, 0.05, n_jobs=6)
mat_acc = time_perm_cluster(output_0_data_trialAvg_acc, output_1_data_trialAvg_acc, 0.05, n_jobs=6)
mat_parietal = time_perm_cluster(output_0_data_trialAvg_parietal, output_1_data_trialAvg_parietal, 0.05, n_jobs=6)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:33: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_0_data_trialAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data(), axis=0))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:34: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_1_data_trialAvg_list_dlpfc.append(np.nanmean(output_1_epochs.get_data(), axis=0))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_0_data_timeAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data()[:, :, start_idx:end_idx], axis=2))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:37: RuntimeWarning: Mean

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:33: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_0_data_trialAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data(), axis=0))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:34: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_1_data_trialAvg_list_dlpfc.append(np.nanmean(output_1_epochs.get_data(), axis=0))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_0_data_timeAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data()[:, :, start_idx:end_idx], axis=2))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:38: FutureWarning: The c

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:33: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_0_data_trialAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data(), axis=0))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:34: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_1_data_trialAvg_list_dlpfc.append(np.nanmean(output_1_epochs.get_data(), axis=0))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_0_data_timeAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data()[:, :, start_idx:end_idx], axis=2))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:38: FutureWarning: The c

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:33: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_0_data_trialAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data(), axis=0))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:34: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_1_data_trialAvg_list_dlpfc.append(np.nanmean(output_1_epochs.get_data(), axis=0))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  output_0_data_timeAvg_list_dlpfc.append(np.nanmean(output_0_epochs.get_data()[:, :, start_idx:end_idx], axis=2))
C:\Users\jz421\AppData\Local\Temp\ipykernel_16672\1952278925.py:38: FutureWarning: The c

KeyError: 'D0094'

### do window stats  
use the time avg outputs from previous cell  
use fdr correction after comparing output 0 and output 1 for each electrode to get a p-values list  
untested 2/4/24


In [46]:
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests

def perform_analysis_and_fdr_correction(data_timeavg_output_0, data_timeavg_output_1):
    """
    Perform paired t-test for each electrode between two conditions and apply FDR correction,
    ignoring NaN values and reporting the number of NaN pairs.
    """
    p_values = []
    t_stats = []
    nan_counts = []  # To store the count of NaN pairs for each electrode

    # Ensure input data dimensions match
    assert data_timeavg_output_0.shape == data_timeavg_output_1.shape, "Data arrays must have the same shape"

    # Perform paired t-test for each electrode, ignoring NaNs
    for i in range(data_timeavg_output_0.shape[1]):  # Iterate over electrodes
        # Identify NaN pairs
        nan_pairs = np.isnan(data_timeavg_output_0[:, i]) | np.isnan(data_timeavg_output_1[:, i])
        nan_count = np.sum(nan_pairs)
        nan_counts.append(nan_count)
        
        # Remove NaN pairs for valid comparison
        valid_indices = ~nan_pairs
        if np.sum(valid_indices) > 1:  # Ensure there are at least two valid pairs for comparison
            t_stat, p_value = stats.ttest_rel(data_timeavg_output_0[valid_indices, i],
                                              data_timeavg_output_1[valid_indices, i])
            t_stats.append(t_stat)
            p_values.append(p_value)
        else:
            # Append NaNs if not enough data points for a valid test
            t_stats.append(np.nan)
            p_values.append(np.nan)

    # Apply FDR correction
    _, p_vals_adj, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

    # Print out the number of NaN pairs for each electrode
    for i, count in enumerate(nan_counts):
        print(f"Electrode {i+1} had {count} NaN pairs excluded from analysis.")

    return np.array(p_vals_adj), np.array(t_stats), np.array(nan_counts)


In [48]:
# Perform the analysis for DLPFC
p_vals_adj_dlpfc, t_stats_dlpfc, nan_pairs = perform_analysis_and_fdr_correction(output_0_data_timeAvg_dlpfc, output_1_data_timeAvg_dlpfc)

# Perform the analysis for ACC
p_vals_adj_acc, t_stats_acc, nan_pairs = perform_analysis_and_fdr_correction(output_0_data_timeAvg_acc, output_1_data_timeAvg_acc)

# Perform the analysis for parietal
p_vals_adj_parietal, t_stats_parietal, nan_pairs = perform_analysis_and_fdr_correction(output_0_data_timeAvg_parietal, output_1_data_timeAvg_parietal)

Electrode 1 had 4 NaN pairs excluded from analysis.
Electrode 2 had 8 NaN pairs excluded from analysis.
Electrode 3 had 0 NaN pairs excluded from analysis.
Electrode 4 had 4 NaN pairs excluded from analysis.
Electrode 5 had 2 NaN pairs excluded from analysis.
Electrode 6 had 2 NaN pairs excluded from analysis.
Electrode 7 had 0 NaN pairs excluded from analysis.
Electrode 8 had 1 NaN pairs excluded from analysis.
Electrode 9 had 0 NaN pairs excluded from analysis.
Electrode 10 had 0 NaN pairs excluded from analysis.
Electrode 11 had 0 NaN pairs excluded from analysis.
Electrode 12 had 21 NaN pairs excluded from analysis.
Electrode 13 had 1 NaN pairs excluded from analysis.
Electrode 14 had 2 NaN pairs excluded from analysis.
Electrode 15 had 41 NaN pairs excluded from analysis.
Electrode 16 had 3 NaN pairs excluded from analysis.
Electrode 17 had 0 NaN pairs excluded from analysis.
Electrode 1 had 0 NaN pairs excluded from analysis.
Electrode 2 had 0 NaN pairs excluded from analysis.
El

In [ ]:
p_vals_adj_parietal

array([0.94528272, 0.10617744, 0.51649575, 0.46177384, 0.51649575,
       0.23341828, 0.94528272, 0.50897238, 0.99473732, 0.94528272,
       0.94528272, 0.94528272, 0.99473732])

### try to do the stats in a more modular way
this is broken right now don't use it

In [ ]:
# import numpy as np

# def calculate_averages(subjects_mne_objects, subject, output_names, sig_electrodes, start_idx, end_idx):
#     trial_avg_list = []
#     time_avg_list = []

#     for output_name in output_names:
#         epochs = subjects_mne_objects[subject][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
#         trial_avg = np.nanmean(epochs.get_data(), axis=0)
#         time_avg = np.nanmean(epochs.get_data()[:, :, start_idx:end_idx], axis=2)
        
#         trial_avg_list.append(trial_avg)
#         time_avg_list.append(time_avg)
    
#     return trial_avg_list, time_avg_list

# # def concatenate_data(data_list):
# #     return np.concatenate(data_list, axis=0)

# # Initialize lists to store data
# output_0_data_trialAvg_list_dlpfc_test = []
# output_1_data_trialAvg_list_dlpfc_test = []
# output_0_data_trialAvg_list_acc = []
# output_1_data_trialAvg_list_acc = []
# output_0_data_trialAvg_list_parietal = []
# output_1_data_trialAvg_list_parietal = []

# # Initialize lists to store time-averaged data
# output_0_data_timeAvg_list_dlpfc_test = []
# output_1_data_timeAvg_list_dlpfc_test = []
# output_0_data_timeAvg_list_acc = []
# output_1_data_timeAvg_list_acc = []
# output_0_data_timeAvg_list_parietal = []
# output_1_data_timeAvg_list_parietal = []

# start_idx, end_idx = 2048, 3072  # Time window from 0 to 0.5 seconds

# # Processing for each electrode type
# for sub in subjects:
    
#     # Processing for dlPFC electrodes
#     if sig_dlpfc_electrodes_per_subject[sub]:
#         dlpfc_trial_avg, dlpfc_time_avg = calculate_averages(subjects_mne_objects, sub, output_names, sig_dlpfc_electrodes_per_subject[sub], start_idx, end_idx)
#         output_0_data_trialAvg_list_dlpfc_test.append(dlpfc_trial_avg[0])
#         output_1_data_trialAvg_list_dlpfc_test.append(dlpfc_trial_avg[1])
#         output_0_data_timeAvg_list_dlpfc_test.append(dlpfc_time_avg[0])
#         output_1_data_timeAvg_list_dlpfc_test.append(dlpfc_time_avg[1])

#     # Processing for ACC electrodes
#     if sig_acc_electrodes_per_subject[sub]:
#         acc_trial_avg, acc_time_avg = calculate_averages(subjects_mne_objects, sub, output_names, sig_acc_electrodes_per_subject[sub], start_idx, end_idx)
#         output_0_data_trialAvg_list_acc.append(acc_trial_avg[0])
#         output_1_data_trialAvg_list_acc.append(acc_trial_avg[1])
#         output_0_data_timeAvg_list_acc.append(acc_time_avg[0])
#         output_1_data_timeAvg_list_acc.append(acc_time_avg[1])

#     # Processing for Parietal electrodes
#     if sig_parietal_electrodes_per_subject[sub]:
#         parietal_trial_avg, parietal_time_avg = calculate_averages(subjects_mne_objects, sub, output_names, sig_parietal_electrodes_per_subject[sub], start_idx, end_idx)
#         output_0_data_trialAvg_list_parietal.append(parietal_trial_avg[0])
#         output_1_data_trialAvg_list_parietal.append(parietal_trial_avg[1])
#         output_0_data_timeAvg_list_parietal.append(parietal_time_avg[0])
#         output_1_data_timeAvg_list_parietal.append(parietal_time_avg[1])

### plot and QC stats

check if no significant differences..

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(mat_dlpfc)
plt.xlabel('Timepoints')
plt.ylabel('Significance (0 or 1)')
plt.title('Permutation Test Significance Over Time')
plt.show()

try a basic t-test as a sanity check (ok its also not significant)

In [ ]:
# import numpy as np
# from scipy.stats import ttest_ind

# p_values = np.array([ttest_ind(output_0_data_trialAvg[:, tp], output_1_data_trialAvg[:, tp]).pvalue for tp in range(output_0_data_trialAvg.shape[1])])
# significance = p_values < 0.05  # Apply a significance threshold, e.g., p < 0.05

# # Plotting
# plt.figure(figsize=(12, 6))
# plt.plot(range(output_0_data_trialAvg.shape[1]), significance, label='Significance (p < 0.05)')
# plt.xlabel('Time Point')
# plt.ylabel('Significance (True/False)')
# plt.title('Statistical Significance Over Time')
# plt.legend()
# plt.show()


### plot dlpfc inc vs. con, avg across all subjects (or whatever conditions you wanna compare, just use the proper output names)

dlpfc  
use blue and gold for congruent and incongruent  
red and green for repeat and switch
# ONLY RUN EITHER SWITCHINESS OR CONGRUENCY!!!

switchiness

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_dlpfc_switchRepeat_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0063_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_acc_output_0, label='Average dlpfc repeat', color='red')
plt.fill_between(times, overall_average_dlpfc_output_0 - overall_sem_dlpfc_output_0, 
                 overall_average_dlpfc_output_0 + overall_sem_dlpfc_output_0, alpha=0.3, color='red')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_dlpfc_output_1, label='Average dlpfc switch', color='green')
plt.fill_between(times, overall_average_dlpfc_output_1 - overall_sem_dlpfc_output_1, 
                 overall_average_dlpfc_output_1 + overall_sem_dlpfc_output_1, alpha=0.3, color='green')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average dlpfc Signal with Standard Error (switch vs repeat)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


congruency

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_dlpfc_congruency_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0063_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_acc_output_0, label='Average dlpfc congruent', color='blue')
plt.fill_between(times, overall_average_dlpfc_output_0 - overall_sem_dlpfc_output_0, 
                 overall_average_dlpfc_output_0 + overall_sem_dlpfc_output_0, alpha=0.3, color='blue')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_dlpfc_output_1, label='Average dlpfc incongruent', color='gold')
plt.fill_between(times, overall_average_dlpfc_output_1 - overall_sem_dlpfc_output_1, 
                 overall_average_dlpfc_output_1 + overall_sem_dlpfc_output_1, alpha=0.3, color='gold')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average dlpfc Signal with Standard Error (inc vs con)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


acc

switchiness

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_acc_switchRepeat_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0063_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_acc_output_0, label='Average acc repeat', color='red')
plt.fill_between(times, overall_average_acc_output_0 - overall_sem_acc_output_0, 
                 overall_average_acc_output_0 + overall_sem_acc_output_0, alpha=0.3, color='red')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_acc_output_1, label='Average acc switch', color='green')
plt.fill_between(times, overall_average_acc_output_1 - overall_sem_acc_output_1, 
                 overall_average_acc_output_1 + overall_sem_acc_output_1, alpha=0.3, color='green')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average acc Signal with Standard Error (switch vs repeat)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


congruency

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_acc_congruency_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0063_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_acc_output_0, label='Average acc congruent', color='blue')
plt.fill_between(times, overall_average_acc_output_0 - overall_sem_acc_output_0, 
                 overall_average_acc_output_0 + overall_sem_acc_output_0, alpha=0.3, color='blue')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_acc_output_1, label='Average acc incongruent', color='gold')
plt.fill_between(times, overall_average_acc_output_1 - overall_sem_acc_output_1, 
                 overall_average_acc_output_1 + overall_sem_acc_output_1, alpha=0.3, color='gold')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average acc Signal with Standard Error (inc vs con)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


parietal

switchiness

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_parietal_switchRepeat_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0063_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_parietal_output_0, label='Average parietal repeat', color='red')
plt.fill_between(times, overall_average_parietal_output_0 - overall_sem_parietal_output_0, 
                 overall_average_parietal_output_0 + overall_sem_parietal_output_0, alpha=0.3, color='red')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_parietal_output_1, label='Average parietal switch', color='green')
plt.fill_between(times, overall_average_parietal_output_1 - overall_sem_parietal_output_1, 
                 overall_average_parietal_output_1 + overall_sem_parietal_output_1, alpha=0.3, color='green')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average parietal Signal with Standard Error (switch vs repeat)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


congruency

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_parietal_congruency_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0063_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_parietal_output_0, label='Average parietal congruent', color='blue')
plt.fill_between(times, overall_average_parietal_output_0 - overall_sem_parietal_output_0, 
                 overall_average_parietal_output_0 + overall_sem_parietal_output_0, alpha=0.3, color='blue')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_parietal_output_1, label='Average parietal incongruent', color='gold')
plt.fill_between(times, overall_average_parietal_output_1 - overall_sem_parietal_output_1, 
                 overall_average_parietal_output_1 + overall_sem_parietal_output_1, alpha=0.3, color='gold')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average parietal Signal with Standard Error (inc vs con)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


### plot individual electrodes

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

firstColor, secondColor= 'red', 'green' #switchiness
# firstColor, secondColor= 'blue', 'gold' #congruency

def plot_electrodes_grid(electrodes_data, grid_num, save_dir, roi, output_names, times, firstColor, secondColor):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        ax.plot(times, data['output_0'], label=f'{roi} {output_names[0]}', color=firstColor)
        ax.fill_between(times, 
                        data['output_0'] - np.std(data['output_0'], ddof=1) / np.sqrt(len(data['output_0'])),
                        data['output_0'] + np.std(data['output_0'], ddof=1) / np.sqrt(len(data['output_0'])), alpha=0.3)
        ax.plot(times, data['output_1'], label=f'{roi} {output_names[1]}', color=secondColor)
        ax.fill_between(times, 
                        data['output_1'] - np.std(data['output_1'], ddof=1) / np.sqrt(len(data['output_1'])),
                        data['output_1'] + np.std(data['output_1'], ddof=1) / np.sqrt(len(data['output_1'])), alpha=0.3)
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Z-score')

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{output_names[0]}_{output_names[1]}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()

# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0



for sub in subjects:
    if sub in sig_dlpfc_electrodes_per_subject:
        for electrode in sig_dlpfc_electrodes_per_subject[sub]:
            electrode_data = {
                'output_0': output_0_data_trialAvg_dlpfc[electrode_counter],
                'output_1': output_1_data_trialAvg_dlpfc[electrode_counter]
            }
            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'dlPFC', output_names, times, firstColor, secondColor)
                
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'dlPFC', output_names, times, firstColor, secondColor)



acc

In [ ]:
# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0

for sub in subjects:
    if sub in sig_acc_electrodes_per_subject:
        for electrode in sig_acc_electrodes_per_subject[sub]:
            electrode_data = {
                'output_0': output_0_data_trialAvg_acc[electrode_counter],
                'output_1': output_1_data_trialAvg_acc[electrode_counter]
            }
            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'acc', output_names, times, firstColor, secondColor)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'acc', output_names, times, firstColor, secondColor)


parietal

In [ ]:
# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0

for sub in subjects:
    if sub in sig_parietal_electrodes_per_subject:
        for electrode in sig_parietal_electrodes_per_subject[sub]:
            electrode_data = {
                'output_0': output_0_data_trialAvg_parietal[electrode_counter],
                'output_1': output_1_data_trialAvg_parietal[electrode_counter]
            }
            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'parietal', output_names, times, firstColor, secondColor)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'parietal', output_names, times, firstColor, secondColor)
